In [29]:
import os
os.chdir('/home/jagust/dino/surf')
from surf import *
%matplotlib inline

In [30]:
young_neg_mat = np.load('/home/jagust/dino/surf/mats/young_neg_med_mat.npy')
young_pos_mat = np.load('/home/jagust/dino/surf/mats/young_pos_med_mat.npy')
young_mat = np.load('/home/jagust/dino/surf/mats/young_med_mat.npy')
old_neg_mat = np.load('/home/jagust/dino/surf/mats/ignore_pib_pos/neg_med_mat.npy')
old_pos_mat = np.load('/home/jagust/dino/surf/mats/ignore_pib_pos/pos_med_mat.npy')
matches = np.load('/home/jagust/dino/surf/matches.npy')

In [105]:
def energy(mat):
    eigs = np.linalg.eig(mat)[0]
    eigs = [abs(eig) for eig in eigs]
    return np.sum(eigs)
def core(g, k):
    return nx.k_core(g, k)

def num_disconnects(g):
    ''' Finds number of disconnected nodes '''
    count = 0
    for subg in nx.connected_component_subgraphs(g):
        if len(subg.nodes()) == 1:
            count += 1
    return count

In [91]:
pos_mat.shape == (0, 0)

True

In [124]:
c = 0.20
k = 10
pos_energy, pos_discs, neg_energy, neg_discs = [], [], [], []
# for c in [.10, .15, .20, .25,.30, .35, .40, .45, .50]:
for match in matches:
    pos_mat, neg_mat = np.load(str(match[0])), np.load(str(match[1]))
    pos_mat, neg_mat = util.threshold_adjacency_matrix(pos_mat, cost=c)[0], util.threshold_adjacency_matrix(neg_mat, cost=c)[0]
    pos_g, neg_g = nx.Graph(pos_mat), nx.Graph(neg_mat)
    pos_discs.append(num_disconnects(pos_g))
    neg_discs.append(num_disconnects(neg_g))
    pos_g, neg_g = core(nx.Graph(pos_mat), k), core(nx.Graph(neg_mat), k)
    pos_mat, neg_mat = nx.to_numpy_matrix(pos_g), nx.to_numpy_matrix(neg_g)
    pos_energy.append(energy(pos_mat))
    neg_energy.append(energy(neg_mat))
print c, k
print stats.ttest_ind(pos_energy, neg_energy)
print stats.ttest_ind(pos_discs, neg_discs)

0.2 10
(1.4148853538375936, 0.16383621348537297)
(-0.58590407242956777, 0.56080389203883385)


In [79]:
young_neg_paths = np.load('/home/jagust/dino/surf/mats/young_geno_neg.npy')
young_pos_paths = np.load('/home/jagust/dino/surf/mats/young_geno_pos.npy')
# young_neg_paths = young_neg_paths[0:10]

In [119]:
young_pos_energy, young_neg_energy, pos_discs, neg_discs = [], [], [], []
c = .2
k = 10
for path in young_neg_paths:
#     print path
    mat = np.load(path)
    mat = np.nan_to_num(mat)
    mat = util.threshold_adjacency_matrix(mat, cost=c)[0]
    neg_g = nx.Graph(mat)
    neg_discs.append(num_disconnects(neg_g))
    g = core(nx.Graph(mat), k)
    mat = nx.to_numpy_matrix(g)
    young_neg_energy.append(energy(mat))
for path in young_pos_paths:
#     print path
    mat = np.load(path)
    mat = np.nan_to_num(mat)
    mat = util.threshold_adjacency_matrix(mat, cost=c)[0]
    pos_g = nx.Graph(mat)
    pos_discs.append(num_disconnects(pos_g))
    g = core(nx.Graph(mat), k)
    mat = nx.to_numpy_matrix(g)
    young_pos_energy.append(energy(mat))
print stats.ttest_ind(young_pos_energy, young_neg_energy)
print stats.ttest_ind(pos_discs, neg_discs)

(-0.80616769789216758, 0.42503480841931218)
(-0.71949321249562659, 0.47612673405808548)


In [81]:
print stats.ttest_ind(young_neg_energy, neg_energy)

(-0.87801428855630614, 0.38389900732632476)


In [82]:
print stats.ttest_ind(young_pos_energy, pos_energy)

(-1.9282707361658329, 0.062734188202645536)


In [ ]:
# Number of Disconnected components